<a href="https://colab.research.google.com/github/sljm12/machine_learning_notebooks/blob/master/Zeroshot_learning_using_BART_and_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is to try Zeroshot learning on News Articles Headings.

The news headings are from IPTC Media Topics https://iptc.org/standards/media-topics/

Baseline code from https://joeddav.github.io/blog/2020/05/29/ZSL.html

In [42]:
!pip -qq install transformers

# Data Prep

In [43]:
!wget https://www.iptc.org/std/NewsCodes/IPTC-MediaTopic-NewsCodes.xlsx

--2020-11-20 08:27:05--  https://www.iptc.org/std/NewsCodes/IPTC-MediaTopic-NewsCodes.xlsx
Resolving www.iptc.org (www.iptc.org)... 194.232.153.3
Connecting to www.iptc.org (www.iptc.org)|194.232.153.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628997 (614K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘IPTC-MediaTopic-NewsCodes.xlsx.1’

IPTC-MediaTopic-New 100%[===================>] 614.25K  1.01MB/s    in 0.6s    

2020-11-20 08:27:07 (1.01 MB/s) - ‘IPTC-MediaTopic-NewsCodes.xlsx.1’ saved [628997/628997]



In [44]:
import pandas as pd
df=pd.read_excel("IPTC-MediaTopic-NewsCodes.xlsx", skiprows=1)

In [45]:
df.head()

,NewsCode-URI,NewsCode-QCode (flat),Level1/NewsCode,Level2/NewsCode,Level3/NewsCode,Level4/NewsCode,Level5/NewsCode,Level6/NewsCode,RetiredDate,Name (ar),Definition (ar),Name (de),Definition (de),Name (dk),Definition (dk),Name (en-GB),Definition (en-GB),Name (es),Definition (es),Name (fr),Definition (fr),Name (no),Definition (no),Name (pt-PT),Definition (pt-PT),Name (pt-BR),Definition (pt-BR),Name (se),Definition (se),Name (zh-Hans),Definition (zh-Hans),SubjectCode mapping,Wikidata mapping
0,http://cv.iptc.org/newscodes/mediatopic/01000000,medtop:01000000,medtop:01000000,NaN,NaN,NaN,NaN,NaN,NaN,فنون، ثقافة وترفيه,NaN,"Kultur, Kunst, Unterhaltung und Medien","Alle Formen von Kunst, Unterhaltung, Kulturerb...","kunst, kultur, underholdning og medier","Kunst, kultur, underholdning og medier.","arts, culture, entertainment and media","All forms of arts, entertainment, cultural her...","Artes, cultura, entretenimiento y medios","Todas las formas de arte, entretenimiento cult...","Arts, culture, divertissement et médias","Toutes les formes d'arts, de divertissement, d...",kultur og underholdning,NaN,"artes, cultura, entretenimento e média","Todas as formas de arte, entretenimento, heran...","artes, cultura, entretenimento e mídia","Todas as formas de arte, entretenimento, heran...","Konst, kultur och nöje",Ämnen som rör utveckling och förfining av det ...,艺术、文化、娱乐和媒体,各种形式的艺术、娱乐、文化遗产和媒体,http://cv.iptc.org/newscodes/subjectcode/01000000,NaN
1,http://cv.iptc.org/newscodes/mediatopic/20000002,medtop:20000002,NaN,medtop:20000002,NaN,NaN,NaN,NaN,NaN,فنون وتسلية,NaN,Kunst und Unterhaltung,Alle Formen von Kunst und Unterhaltung,kunst og underholdning,Alle former for kunst og underholdning.,arts and entertainment,All forms of arts and entertainment,Arte y entretenimiento,Todas las formas de arte y de entretenimiento,Arts et divertissement,Toutes les formes d'arts et de divertissement,kunst og underholdning,NaN,arte e entretenimento,Todas as formas de arte e entretenimento.,arte e entretenimento,Todas as formas de arte e entretenimento.,Konst och underhållning,Alla former av konst och underhållning som int...,艺术与娱乐,各种形式的艺术和娱乐,NaN,https://www.wikidata.org/entity/Q2018526
2,http://cv.iptc.org/newscodes/mediatopic/20000003,medtop:20000003,NaN,NaN,medtop:20000003,NaN,NaN,NaN,NaN,تحريك,NaN,Animation,"Erzählung in Form von Computergrafik, als Lang...",animation,Historier fortalt gennem animerede tegninger e...,animation,Stories told through animated drawings in eith...,Animación,Historias contadas a través de dibujos animado...,Dessin animé,Histoires racontées à travers des dessins anim...,animasjonsfilm,NaN,animação,Técnica que permite dar a desenhos ou bonecos ...,animação,"Animação, incluindo longas e curtas-metragens,...",Animation,"Rörlig bild skapad av en serie teckningar, ler...",动画,长短篇的动画故事,http://cv.iptc.org/newscodes/subjectcode/01025000,https://www.wikidata.org/entity/Q11425
3,http://cv.iptc.org/newscodes/mediatopic/20001135,medtop:20001135,NaN,NaN,medtop:20001135,NaN,NaN,NaN,NaN,NaN,NaN,Kunstausstellung,Zeitlich begrenzte Präsentation von Kunst in e...,udstilling,Midlertidig præsentation af kunst eller særemn...,art exhibition,"Temporary presentation of art in museums, art ...",Exposición de arte,"Presentación temporal de arte en museos, salas...",Exposition artistique,"Présentation temporaire d'art dans des musées,...",kunstutstilling,NaN,exposição artística,"Apresentação temporária de arte em museus, sal...",exposição artística,"Apresentação temporária de arte em museus, sal...",Konstutställning,Ett tillfälle då en eller flera konstverk visa...,艺术展览,在博物馆、艺术大厅或画廊举办的短期艺术品展出,NaN,https://www.wikidata.org/entity/Q667276
4,http://cv.iptc.org/newscodes/mediatopic/20000004,medtop:20000004,NaN,NaN,medtop:20000004,NaN,NaN,NaN,NaN,كرتون,NaN,Comic,"Zeichnungen, wie Karikaturen und Comics, die o...",tegneserie,"Tegninger, både som underholding og redaktione...",cartoon,"Drawings, such as editorial cartoons and comic...",Caricaturas,"Dibujos, como 

In [46]:
# Get the 2nd level headings because they are more detailed that the first level headings
df_top_level = df[df["Level2/NewsCode"].notnull()]

In [47]:
df_top_level.size

3597

In [48]:
topics = [i for i in df_top_level["Name (en-GB)"]]

# Zeroshot learning portion

In [49]:
from transformers import BartForSequenceClassification, BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')

# pose sequence as a NLI premise and label (politics) as a hypothesis
premise = 'Who are you voting for in 2020?'
hypothesis = 'This text is about politics.'



Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
def predict(tokenizer, model, premise, category):
  hypothesis = "This text is about "+category
  input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt')
  #print(input_ids)
  logits = model(input_ids)[0]

  #print(logits)
  entail_contradiction_logits = logits[:,[0,2]]
  #print(entail_contradiction_logits)
  probs = entail_contradiction_logits.softmax(dim=1)
  true_prob = probs[:,1].item() * 100
  #print(f'Probability that the label is true: {true_prob:0.2f}%')
  return true_prob

In [51]:
results = []
for i in topics:
  prob = predict(tokenizer, model, premise, i)
  results.append({
      "category":i,
      "prob":prob
  })

arts and entertainment 0.7709584198892117
culture 3.4023717045783997
mass media 1.4756249263882637
act of terror 0.5168270319700241
armed conflict 0.677485391497612
civil unrest 1.8349843099713326
coup d'etat 0.2247572410851717
massacre 0.6081278901547194
peace process 1.1101458221673965
post-war reconstruction 0.861875806003809
prisoners of war 0.7250878494232893
crime 1.507717277854681
judiciary 2.858201786875725
justice 2.6447972282767296
law 1.9982187077403069
law enforcement 0.5001340061426163
accident and emergency incident 0.49067186191678047
disaster 1.5637805685400963
emergency incident 0.5152593832463026
emergency planning 0.10806771460920572
emergency response 0.2357398858293891
business information 1.0673231445252895
economic sector 4.671101272106171
economy 2.7262821793556213
market and exchange 1.3947543688118458
parent organisation 2.4293044582009315
religious education 0.23001451045274734
school 0.40059909224510193
social learning 0.9947064332664013
teaching and learnin

In [52]:
result_frame = pd.DataFrame(data=results)

In [53]:
result_frame.sort_values("prob",ascending=False)

,category,prob
60,election,97.019875
68,political process,71.554357
62,government,59.943539
48,people,38.544866
63,government policy,38.055763
...,...,...
57,exercise and fitness,0.106452
108,weather warning,0.086388
96,bodybuilding,0.078919
106,weather phenomena,0.075119


# Original Code

In [ ]:
# run through model pre-trained on MNLI
input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt')
print(input_ids)
logits = model(input_ids)[0]

print(logits)


tensor([[    0, 12375,    32,    47,  3434,    13,    11,  2760,   116,     2,
             2,   713,  2788,    16,    59,  2302,     4,     2]])
tensor([[-2.5443,  0.8770,  1.3904]], grad_fn=<AddmmBackward>)


In [ ]:
# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,2]]
print(entail_contradiction_logits)
probs = entail_contradiction_logits.softmax(dim=1)
true_prob = probs[:,1].item() * 100
print(f'Probability that the label is true: {true_prob:0.2f}%')

tensor([[-2.5443,  1.3904]], grad_fn=<IndexBackward>)
Probability that the label is true: 98.08%


In [ ]:
logits[:,[0,1]]

tensor([[-2.5443,  0.8770]], grad_fn=<IndexBackward>)

In [ ]:
probs

tensor([[0.0192, 0.9808]], grad_fn=<SoftmaxBackward>)